# What is Featuristic?

![featuristic_logo](_static/logo.png "Featuristic")

**Featuristic** uses Genetic Algorithms to automate the process of **feature engineering** and **feature selection**, enhancing the performance of machine learning models by optimizing their predictive capabilities.

## Understanding Genetic Feature Synthesis

Featuristic uses a technique known as symbolic regression to intelligently generate interpretable mathematical formulas, which are then used to construct new features from your dataset.

Initially, Featuristic creates a diverse population of formulas using fundamental mathematical operators such as `add`, `subtract`, `sin`, `tan`, `square`, `sqrt`, and more.

For instance, a formula generated by Featuristic might look like this: `(abs(square(feature_1)) - feature_2) * feature_3`.

Next, Featuristic evaluates the performance of these formulas by measuring how well the features they generate correlate with the target variable. The formulas that produce features with the highest correlation are then selected and combined using a genetic algorithm to create offspring, as shown below.

![Symbolic Regression Example](_static/symbolic_regression_example.png "Symbolic Regression Example")

These offspring may also undergo point mutations, altering specific operators within the formula, as illustrated below.

![Mutation Example](_static/mutation_example.png "Mutation Example")

This iterative process continues across multiple generations, continually refining the population of formulas with the goal of generating features that exhibit strong correlations with the target variable.

## Quickstart

Below is a simple example of how Featuristic performs automated feature engineering and selection on the widely used `cars` dataset.

Featuristic operates in two distinct steps:

1. **Genetic Feature Synthesis:** In this initial phase, Featuristic intelligently evolves new features through a form of symbolic regression. This involves the generation of mathematical expressions using genetic algorithms. These expressions are designed to capture complex relationships within the dataset.

2. **Genetic Feature Selection:** Following the creation of new features, Featuristic employs a Genetic Feature Selection algorithm. This algorithm searches through the newly formed feature space to identify the most optimal subset of features. The objective is to maximize predictive accuracy while minimizing the number of features required for model training.

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error
import featuristic as ft
import numpy as np

np.random.seed(8888)

print(ft.__version__)

0.1.1


### Load the Data

In [2]:
X, y = ft.fetch_cars_dataset()

X.head()

,displacement,cylinders,horsepower,weight,acceleration,model_year,origin
0,307.0,8,130.0,3504,12.0,70,1
1,350.0,8,165.0,3693,11.5,70,1
2,318.0,8,150.0,3436,11.0,70,1
3,304.0,8,150.0,3433,12.0,70,1
4,302.0,8,140.0,3449,10.5,70,1


In [3]:
y.head()

0    18.0
1    15.0
2    18.0
3    16.0
4    17.0
Name: mpg, dtype: float64

### Genetic Feature Synthesis

Now, let's dive into the exciting part: running the Genetic Feature Synthesis to automatically engineer new features from our dataset!

Before we proceed, it's important to ensure a robust evaluation of our model's performance. To achieve this, we'll first split our dataset into training and testing sets. The training set will be used to train our model, while the testing set will remain unseen during the training process and will serve as an independent dataset to evaluate the model's performance.

Once our data is appropriately split, we'll initiate the Genetic Feature Synthesis process. We've configured the genetic algorithm to  synthesize 10 new features for us. This entails evolving a population consisting of 100 individuals iteratively over 50 generations. To ensure optimal performance, we've set the genetic algorithm to halt early if it fails to improve upon the best feature identified within 25 generations. Additionally, for enhanced computational efficiency, we've designated `n_jobs` as -1, enabling concurrent execution across all available CPUs on our computer.

With everything set up, we simply call the `fit` function to generate our new features.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

synth = ft.GeneticFeatureSynthesis(
    num_features=10,
    population_size=100,
    max_generations=50,
    early_termination_iters=25,
    n_jobs=-1,
)

synth.fit(X_train, y_train)

None

Creating new features...:  48%|████████████████████████████████████████▊                                            | 24/50 [00:06<00:05,  4.89it/s]/Users/martin/repos/featuristic/.venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sin
  result = getattr(ufunc, method)(*inputs, **kwargs)


ValueError: array must not contain infs or NaNs

Next, we call the `transform` function to generate a dataframe containing our new features. By default, the `GeneticFeatureSynthesis` class will return both the original features and the newly synthesised features. However, we return just the new features by setting the `return_all_features` argument to False when we create the class. 

We can also combine both the `fit` and `transform` steps into one step by calling `fit_transform` instead.

In [ ]:
features = synth.transform(X_train)

features.head()

Our newly engineered features currently have generic names. However, since Featuristic synthesizes these features by the applying mathematical expressions to the data, we can look at the underlying formulas responsible for each feature's creation.

In [ ]:
info = synth.get_feature_info()
info[["name", "formula"]].head()

### Feature Selection

Following the synthesis of new features, the next step involves using another genetic algorithm for feature selection. This process sifts through the pool of generated features to identify the subset that optimally contributes to predictive performance while minimizing redundancy. 

Through iterative refinement, this algorithm aims to strike a balance between predictive power and model complexity, culminating in an optimized feature set integrating into your machine learning models.

### Define the Cost Function

We set up a custom ojective function that the Genetic Feature Selection algorithm will use to quantify how well the subset of features predicts the target. Please note that the function should return a value to minimize so a smaller value is better. If you want to maximize a metric, you should multiply the output of your objective_function by -1.

In [ ]:
def objective_function(X, y):
    model = LinearRegression()
    scores = cross_val_score(model, X, y, cv=3, scoring="neg_mean_absolute_error")
    return scores.mean()

Next, we set up the Genetic Feature Selector. We've configured the genetic algorithm to evolve a population consisting of 100 individuals iteratively over 50 generations. To ensure optimal performance, we've set the genetic algorithm to halt early if it fails to improve upon the best feature set identified within 15 generations. Additionally, for enhanced computational efficiency, we've designated `n_jobs` as -1, enabling concurrent execution across all available CPUs on our computer.

In [ ]:
selector = ft.GeneticFeatureSelector(
    objective_function,
    population_size = 100,
    max_generations = 50,
    early_termination_iters = 15,
    n_jobs = -1,
)

selector.fit(features, y_train)

selected_features = selector.transform(features)

### The New Features

Let's print out our new features to see what was generated for us. You can see that featurize has kept three of the original features ("displacement", "cylinders", "origin") and has kept four of the features created via the Genetic Feature Synthesis.

In [ ]:
selected_features.head()

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)
original_mae = mean_absolute_error(y_test, preds)
original_mae

In [ ]:
model = LinearRegression()
model.fit(features, y_train)

test_features = synth.transform(X_test)
preds = model.predict(test_features)
featurized_mae = mean_absolute_error(y_test, preds)
featurized_mae

In [ ]:
print(
    f"Original MAE: {original_mae}, Featuristic MAE: {featurized_mae}, \
    Improvement: {round((1 - (featurized_mae / original_mae))* 100, 1)}%"
)